In [1]:
#@title Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


# Part 1: Load/Analyze Data

##What is your dataset? Please describe your dataset and where you got it

My dataset is from https://github.com/bananathrowingmachine/FastPartitionExperimentDocs/tree/main/Previous%20Results which is some saved data generated from a personal project a wrote in May, which I made just to see if I could make a really fast algorith that solves the psuedo-polynomial time partition problem, at least compared to the solution found on Wikipedia (which I did infact beat quite well). The data essentially has 3 x variables and a y variable, where the x variables are the algorithm type (with there being 4 types, the Wikipedia solution, one change I made, another change I made, and then both changes combined together), the amount of integers in the set, and the approximate sum of the set and then the y variable is the average amount of iterations each method took to solve the same randomly(-ish) generated set. For more information on how the experiment was run as a whole you can look here: https://github.com/bananathrowingmachine/FastPartitionExperiment

For this project the data is just linked directly to where it's stored on GitHub with the argument raw=true so that it doesn't redirect to the human visiable page.

How I convert the data is I will turn every combination of inputs and results into a single long list, where the first variable will denote if the data comes from a Memoized algorithm (signified as 1) or a Tabulated algorithm (signified as 0), the second variable will denote if the date comes from a "Crazy" algorithm (signified as 1, with the main GitHub repo elaborating on what "Crazy" means), or a "Normal" algorithm (signified as 0), the third variable will be the "Sum Target" (also elaborated on in the main GitHub repository) and the fourth variable will be the integer count.

Due to my data being available publicly on my Github, the direct links to the sheets already in the code are sufficient for accessing the necessary data frames and vstack them all into a numpy array.

##Load your dataset / Implement your dataloader

In [4]:
def processSheet(fileName: str):
    sheet = pd.ExcelFile(fileName)
    processedData = []
    for sheetName in sheet.sheet_names:
        if sheetName not in ['Recursive Normal', 'Absolute Target Sum']:
            df = pd.read_excel(sheet, sheet_name=sheetName)
            dfReset = df.reset_index().rename(columns={'index': 'Row_Index'})
            dfLong = dfReset.melt(id_vars=['Row_Index'], var_name='Col_Index', value_name='Value')
            dfCleaned = dfLong[dfLong['Col_Index'] != 'Unnamed: 0'].copy()
            dfCleaned['Col_Index'] = pd.to_numeric(dfCleaned['Col_Index']).astype(int)

            rows = dfCleaned.values.shape[0]
            crazyVals = np.ones((rows, 1)) if 'Crazy' in sheetName else np.zeros((rows, 1))
            memoVals = np.ones((rows, 1)) if 'Memoized' in sheetName else np.zeros((rows, 1))
            processedData.append(np.hstack((memoVals, crazyVals, dfCleaned)))

    return np.vstack(processedData)

# Links to the excel sheets directly on my GitHub so I don't have to upload them to Colab.
# These files are public so the data will be converted into a dataframe automatically when this cell is run
data1 = 'https://github.com/bananathrowingmachine/FastPartitionExperimentDocs/blob/main/Previous%20Results/May%2026%2C%202025/data_tables/Results.xlsx?raw=true'
data2 = 'https://github.com/bananathrowingmachine/FastPartitionExperimentDocs/blob/main/Previous%20Results/Jul%2012%2C%202025/data_tables/Results.xlsx?raw=true'
numpyDataArray = np.vstack([processSheet(data1), processSheet(data2)])
x_train, x_combine, y_train, y_combine = train_test_split(numpyDataArray[:, 0:4], numpyDataArray[:, 4], test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_combine, y_combine, test_size=0.5, random_state=42)

##Dataset Analyis Statistics

In [20]:
print('Dataset Partition Sizes:')
print(f'x_train shape: {x_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'x_val shape: {x_val.shape}')
print(f'y_val shape: {y_val.shape}')
print(f'x_test shape: {x_test.shape}')
print(f'y_test shape: {y_test.shape}\n')

print('Example rows:')
print('Memoized Flag | Crazy Flag | Sum Target Index | Integer Count')
print(x_train[0:5])
print('Average iterations:')
print(f'{y_train[0:5]}\n')

print('General Statistics:')
print('The dataset constitutes a full factorial design where all four input features are varied systematically to cover every possible combination of their values.')
print('However do note that due to the code uploading 2 sheets, each combination of features occurs exactly twice, although the corresponding y does have some variation.')
fullLength = x_train.shape[0] + x_val.shape[0] + x_test.shape[0]
print(f'Each value of the \'Memoized\' feature has {fullLength/2} occurances.')
print(f'Each value of the \'Crazy\' feature has {fullLength/2} occurances.')
print(f'Each value of the \'Sum Target Index\' feature has {fullLength/21} occurances.')
print(f'Each value of the \'Integer Count\' feature has {fullLength/20} occurances.')



Dataset Partition Sizes:
x_train shape: (2352, 4)
y_train shape: (2352,)
x_val shape: (504, 4)
y_val shape: (504,)
x_test shape: (504, 4)
y_test shape: (504,)

Example rows:
Memoized Flag | Crazy Flag | Sum Target Index | Integer Count
[[  0.   1.  15.  90.]
 [  0.   0.   0.  40.]
 [  1.   0.  15.  95.]
 [  1.   1.  19.  80.]
 [  0.   0.   8. 100.]]
Average iterations:
[9.94945914e+07 3.99281600e+05 4.73466736e+07 1.52217400e+04
 1.31108548e+08]

General Statistics:
The dataset constitutes a full factorial design where all four input features are varied systematically to cover every possible combination of their values.
However do note that due to the code uploading 2 sheets, each combination of features occurs exactly twice, although the corresponding y does have some variation.
Each value of the 'Memoized' feature has 1680.0 occurances.
Each value of the 'Crazy' feature has 1680.0 occurances.
Each value of the 'Sum Target Index' feature has 160.0 occurances.
Each value of the 'Intege

# Part 2: Literature Review

TODO - Follow instructions in the assignment for your literature review

# Part 3: Implement networks in PyTorch

Note of warning here: Depending on how easily you can allocate GPU resources, you may want to make your network much shallower so that you can train it more easily
Aim to have one gradient update take no more than a few seconds
May also want to reduce the number of training steps if training is too slow

In [ ]:
# TODO - implement your PyTorch Module

In [ ]:
# TODO - implement your training loop

In [ ]:
# TODO - plot train and validation accuracy of your model during training

Hint: look at earlier homeworks / resuse code from those to help you here

# Part 4: Run Hyperparameter Experiments

In [ ]:
# TODO - perform hyperparameter grid searches and plot accuracies

In [ ]:
# TODO - perform other experiments and plot accuracies

In [ ]:
# TODO - calculate the final test accuracy

TODO - Explain your experiments above. See assignment doc for more details.